<a href="https://colab.research.google.com/github/grmurad/GM_SDS-CP022-ai-travel-companion/blob/main/Assisttravel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install packages and setup files for each time need to activate google colab session

!pip install transformers accelerate bitsandbytes
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install geonamescache
!pip install amadeus
!pip install aiohttp
!pip install flask flask-ngrok
!pip install pyngrok
!pip install fuzzywuzzy python-Levenshtein
!pip install googlemaps
import os
import shutil
from google.colab import drive

def setup_colab_environment():
    """
    Sets up the Colab environment by creating folders, mounting Google Drive,
    and copying necessary files.
    """

    # 1. Create folders
    os.makedirs('/content/templates', exist_ok=True)
    os.makedirs('/content/static', exist_ok=True)

    # 2. Mount Google Drive
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')
        print("Google Drive mounted successfully.")
    else:
        print("Google Drive is already mounted.")

    # 3. Define source and destination paths
    # ***VERIFY THIS PATH*** to match your Google Drive folder structure
    drive_base_path = '/content/drive/MyDrive/travel_app/Templates/Deep'
    colab_templates_path = '/content/templates/'
    colab_static_path = '/content/static/'
    colab_base_path = '/content/'


    # 4. Copy files
    # HTML files to templates
    html_files = ['index.html', 'results.html', 'selecionar_cidade.html']  # Replace with your HTML file names
    for file in html_files:
        #This is just for debugging
        print(os.path.join(drive_base_path, file))
        shutil.copy(os.path.join(drive_base_path, file), colab_templates_path)

    # CSV files to /content
    csv_files = ['cidades_filtradas.json', 'IATA_Cities.csv']  # Replace with your CSV file names
    for file in csv_files:
      #This is just for debugging
        print(os.path.join(colab_base_path, file))
        shutil.copy(os.path.join(drive_base_path, file), colab_base_path)

      # GIF file to static
    gif_file = 'HI9M.gif'  # Replace with your GIF file name
    shutil.copy(os.path.join(drive_base_path, gif_file), colab_static_path)

    print("Colab environment setup complete!")

# Call the function to set up the environment
setup_colab_environment()

In [ ]:
from flask import Flask, request, render_template, redirect, url_for, Response
from flask_ngrok import run_with_ngrok
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from accelerate import Accelerator
import torch
from amadeus import Client, ResponseError
from huggingface_hub import login
from google.colab import userdata
import re
import logging
import os
import asyncio
import aiohttp
from datetime import datetime, timedelta
import base64
from pyngrok import ngrok
from fuzzywuzzy import process
import json
import csv
import pandas as pd
import random
import googlemaps

logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)
logger.info("This is a test log")

# Carregar a base de dados de cidades
with open("cidades_filtradas.json", "r", encoding="utf-8") as arquivo:
    BASE_CIDADES = json.load(arquivo)

def carregar_base_iata(caminho_arquivo):
    df = pd.read_csv(caminho_arquivo, on_bad_lines='skip')
    required_columns = ["name", "iso_country", "municipality", "iata_code"]
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Coluna '{col}' não encontrada no arquivo CSV.")
    df = df.dropna(subset=['iata_code'])
    df = df[df['iata_code'] != '']
    df['municipality'] = df['municipality'].fillna('')
    base_iata = df[["name", "iso_country", "municipality", "iata_code"]].rename(columns={
        "name": "nome_aeroporto",
        "iso_country": "pais",
        "municipality": "cidade",
        "iata_code": "iata"
    }).to_dict(orient='records')
    return base_iata

BASE_IATA = carregar_base_iata("IATA_Cities.csv")

import random

# Lista de lugares famosos para turismo
LUGARES_FAMOSOS = [
    "Paris", "New York", "Tokyo", "Rome", "London",
    "Barcelona", "Dubai", "Sydney", "Rio de Janeiro", "Machu Picchu"
]

def buscar_imagem_aleatoria():
    lugar = random.choice(LUGARES_FAMOSOS)
    imagens = buscar_imagens_pixabay(lugar)
    return imagens[0] if imagens else None

    # Para requisições GET, buscar uma imagem aleatória
    imagem_fundo = buscar_imagem_aleatoria()
    return render_template("index.html", imagem_fundo=imagem_fundo)



# Initialize Google Maps client
gmaps = googlemaps.Client(key=userdata.get('Maps'))

def get_address_from_lat_lng(latitude, longitude):
    try:
        # Reverse geocode the latitude and longitude
        reverse_geocode_result = gmaps.reverse_geocode((latitude, longitude))
        if reverse_geocode_result:
            # Extract the formatted address
            return reverse_geocode_result[0]["formatted_address"]
        return "N/A"
    except Exception as e:
        logger.error(f"Erro na API do Google Maps: {e}")
        return "N/A"

def batch_hotel_ids(hotel_ids, batch_size=50):
    """
    Split the list of hotel IDs into smaller batches.
    """
    for i in range(0, len(hotel_ids), batch_size):
        yield hotel_ids[i:i + batch_size]

def buscar_iata_por_cidade(nome_cidade, codigo_pais=None):
    nome_cidade = nome_cidade.strip().lower()
    resultados = []
    for cidade in BASE_IATA:
        cidade_nome = str(cidade.get("cidade", "")).lower()
        if nome_cidade in cidade_nome:
            if codigo_pais and cidade.get("pais", "").lower() != codigo_pais.lower():
                continue
            resultados.append({
                "nome": cidade["cidade"],
                "iata": cidade["iata"]
            })
    return resultados

import requests

PIXABAY_API_KEY = userdata.get('Pixabay')


def buscar_imagens_pixabay(query):
    url = f"https://pixabay.com/api/?key={PIXABAY_API_KEY}&q={query}&image_type=photo&per_page=5"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return [imagem["largeImageURL"] for imagem in data.get("hits", [])]
    return []

ngrok.set_auth_token(userdata.get('NGROK'))
app = Flask(__name__)

# Middleware to add custom User-Agent header
@app.after_request
def add_custom_user_agent(response):
    response.headers['User-Agent'] = 'Custom-Agent'
    return response

public_url = ngrok.connect(5000).public_url

login(token=userdata.get('HUGGINGFACE_TOKEN'))
AMADEUS_CLIENT_ID = userdata.get('AMADEUS_API_KEY')
AMADEUS_CLIENT_SECRET = userdata.get('AMADEUS_API_SECRET')

if not AMADEUS_CLIENT_ID or not AMADEUS_CLIENT_SECRET:
    raise ValueError("Amadeus credentials not found in Colab Secrets. Please add them.")

amadeus = Client(
    client_id=AMADEUS_CLIENT_ID,
    client_secret=AMADEUS_CLIENT_SECRET)

# Restante do código Flask (modelo, funções, rotas, etc.) permanece o mesmo...

async def get_session():
    return aiohttp.ClientSession()

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)
accelerator = Accelerator()
model, tokenizer = accelerator.prepare(model, tokenizer)

def validate_date(date_str):
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        return True
    except ValueError:
        return False

def extrair_cidades_e_iata(texto):
    padrao = r"([A-Za-zÀ-ÿ,\s]+)\s*\(([A-Z]{3})\)"
    matches = re.findall(padrao, texto)
    cidades = [{"nome": cidade.title().strip(), "iata": iata.strip()} for cidade, iata in matches]
    return cidades

def buscar_cidades_similares_local(nome_cidade, limite=5):
    nomes_cidades = [cidade["nome"] for cidade in BASE_CIDADES]
    resultados = process.extract(nome_cidade, nomes_cidades, limit=limite)
    cidades_similares = []
    for nome, score in resultados:
        if score >= 70:
            cidade = next((c for c in BASE_CIDADES if c["nome"] == nome), None)
            if cidade:
                cidades_similares.append(cidade)
    return cidades_similares

async def buscar_hoteis_por_cidade(session, cidade_iata, access_token):
    try:
        async with session.get(
            "https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city",
            params={"cityCode": cidade_iata},
            headers={"Authorization": f"Bearer {access_token}"}
        ) as response:
            if response.status == 200:
                hotel_data = (await response.json()).get("data", [])
                if not hotel_data:
                    return "No hotels found for this city."

                # Extract hotel IDs and names
                hotel_list = [{"id": hotel.get("hotelId", "N/A"), "name": hotel.get("name", "N/A")} for hotel in hotel_data]
                print(hotel_list)
                return hotel_list
            else:
                error_response = await response.text()
                logger.error(f"Failed to fetch hotels: {response.status}, Response: {error_response}")
                return None
    except Exception as e:
        logger.error(f" Buscar_hoteis: Erro na API Amadeus: {e}")
        return None


async def buscar_ofertas_multiplos_hoteis(session, hotel_ids, access_token, check_in_date, check_out_date):
    try:
        detailed_hotels = []

        # Split hotel IDs into batches
        for batch in batch_hotel_ids(hotel_ids):
            params = {
                "hotelIds": ",".join(batch),  # Convert batch of hotel IDs to a comma-separated string
                "checkInDate": check_in_date,
                "checkOutDate": check_out_date,
                "roomQuantity": 1,  # Number of rooms
                "adults": 1,  # Number of adults
                "currency": "USD",  # Currency for pricing
                "bestRateOnly": "false",  # Allow multiple offers per hotel
                "includeClosed": "false",  # Exclude closed hotels
                "lang": "EN"  # Language for responses
            }

            async with session.get(
                "https://test.api.amadeus.com/v3/shopping/hotel-offers",
                params=params,
                headers={"Authorization": f"Bearer {access_token}"}
            ) as response:
                if response.status == 200:
                    hotel_data = (await response.json()).get("data", [])
                    if not hotel_data:
                        continue

                    # Extract details for each hotel
                    for hotel in hotel_data:
                        hotel_name = hotel.get("hotel", {}).get("name", "N/A")
                        hotel_id = hotel.get("hotel", {}).get("hotelId", "N/A")
                        latitude = hotel.get("hotel", {}).get("latitude", "N/A")
                        longitude = hotel.get("hotel", {}).get("longitude", "N/A")
                        offers = hotel.get("offers", [])

                        # Get full address using Google Maps Geocoding API
                        endereco = get_address_from_lat_lng(latitude, longitude)

                        # Extract price details (if available)
                        preco = "N/A"
                        if offers:
                            preco = offers[0].get("price", {}).get("total", "N/A")

                        hotel_info = {
                            "nome": hotel_name,
                            "id": hotel_id,
                            "endereco": endereco,
                            "preco": preco
                        }
                        detailed_hotels.append(hotel_info)
                else:
                    error_response = await response.text()
                    logger.error(f"Failed to fetch hotel offers: {response.status}, Response: {error_response}")
                    continue

        return detailed_hotels
    except Exception as e:
        logger.error(f"Erro na API Amadeus: {e}")
        return None

async def buscar_detalhes_hotel(session, hotel_id, access_token):
    try:
        async with session.get(
            f"https://test.api.amadeus.com/v1/reference-data/locations/hotels/{hotel_id}",
            headers={"Authorization": f"Bearer {access_token}"}
        ) as response:
            if response.status == 200:
                hotel_details = await response.json()
                return hotel_details.get("data", {})
            else:
                logger.error(f"Failed to fetch hotel details: {response.status}")
                return None
    except Exception as e:
        logger.error(f"Buscar Detalhes Hotel: Erro na API Amadeus: {e}")
        return None

def get_default_departure_date():
    today = datetime.today()
    default_date = today + timedelta(days=10)
    return default_date.strftime("%Y-%m-%d")

def gerar_sugestoes_viagem(prompt_usuario, num_sugestoes=3):
    prompt_usuario += (
        f" Liste {num_sugestoes} de destinos turísticos, incluindo:"
        f" - Informe cidades com serviço regular de voos que sejam proximas às atracoes destacadas: Nome da Cidade e seu codigo IATA em parentesis: (exemplo: 'San Juan (SJU)')."
        f" - Principais atrações e atividades. use no maximo 1000 palavras de forma que resuma bem o pedido do usuario"
    )
    inputs = tokenizer(prompt_usuario, return_tensors="pt").to("cuda")
    inputs["attention_mask"] = torch.ones_like(inputs["input_ids"]).to("cuda")
    output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=1000,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(output[0], skip_special_tokens=True).strip()
    print(response)
    response = response.replace(prompt_usuario, "").strip()
    return response, extrair_cidades_e_iata(response)

async def buscar_hoteis(session, cidade_iata, access_token):
    try:
        async with session.get(
            "https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city",
            params={"cityCode": cidade_iata},
            headers={"Authorization": f"Bearer {access_token}"}
        ) as response:
            if response.status == 200:
                hotel_data = (await response.json()).get("data", [])
                if not hotel_data:
                    return "No hotels found for this city."

                hotel_list = []
                for hotel in hotel_data[:5]:  # Limita a 5 hotéis
                    hotel_id = hotel.get("hotelId")
                    hotel_name = hotel.get("name", "N/A")

                    # Buscar detalhes do hotel
                    hotel_details = await buscar_detalhes_hotel(session, hotel_id, access_token)
                    if hotel_details:
                        endereco = hotel_details.get("address", {}).get("lines", ["N/A"])[0]
                        telefone = hotel_details.get("contact", {}).get("phone", "N/A")
                        preco = hotel_details.get("price", {}).get("amount", "N/A")
                    else:
                        endereco = "N/A"
                        telefone = "N/A"
                        preco = "N/A"

                    hotel_info = {
                        "nome": hotel_name,
                        "id": hotel_id,
                        "endereco": endereco,
                        "telefone": telefone,
                        "preco": preco
                    }
                    hotel_list.append(hotel_info)

                return hotel_list
            else:
                logger.error(f"Failed to fetch hotels: {response.status}")
                return None
    except Exception as e:
        logger.error(f"Buscar_Hoteis: Erro na API Amadeus: {e}")
        return None

async def buscar_voos(session, origem_iata, destino, access_token, data_partida="2025-03-10", adultos=1):
    try:
        params = {
            "originLocationCode": origem_iata,
            "destinationLocationCode": destino,
            "departureDate": data_partida,
            "adults": adultos,
            "max": 5  # Limit to 5 results
        }
        async with session.get(
            "https://test.api.amadeus.com/v2/shopping/flight-offers",
            params=params,
            headers={"Authorization": f"Bearer {access_token}"}
        ) as response:
            if response.status == 200:
                flight_data = (await response.json()).get("data", [])
                if not flight_data:
                    logger.warning(f"No flight data found for {origem_iata} -> {destino}")
                    return None  # Retorna None se não houver voos

                # Fetch airline names
                airline_codes = set()
                for flight in flight_data:
                    for segment in flight["itineraries"][0]["segments"]:
                        airline_codes.add(segment["carrierCode"])

                # Map carrier codes to airline names
                airline_names = {}
                for code in airline_codes:
                    async with session.get(
                        "https://test.api.amadeus.com/v1/reference-data/airlines",
                        params={"airlineCodes": code},
                        headers={"Authorization": f"Bearer {access_token}"}
                    ) as airline_response:
                        if airline_response.status == 200:
                            airline_info = (await airline_response.json()).get("data", [])
                            if airline_info:
                                airline_names[code] = airline_info[0].get("businessName", code)

                # Format the flight list with more details
                flights = []
                for flight in flight_data:
                    segments = flight["itineraries"][0]["segments"]
                    flight_info = {
                        "airline": airline_names.get(segments[0].get("carrierCode", "N/A"), "N/A"),
                        "flightNumber": segments[0].get("number", "N/A"),
                        "origin": segments[0].get("departure", {}).get("iataCode", "N/A"),
                        "destination": segments[-1].get("arrival", {}).get("iataCode", "N/A"),
                        "departureTime": segments[0].get("departure", {}).get("at", "N/A"),
                        "arrivalTime": segments[-1].get("arrival", {}).get("at", "N/A"),
                        "price": flight["price"].get("total", "N/A"),
                        "currency": flight["price"].get("currency", "N/A")
                    }
                    flights.append(flight_info)
                return flights
            else:
                error_response = await response.text()
                logger.error(f"Failed to fetch flights: {response.status}, Response: {error_response}")
                return None  # Retorna None em caso de erro
    except Exception as e:
        logger.error(f"Buscar_Voos : Erro na API Amadeus: {e}")
        return None  # Retorna None em caso de exceção

async def get_amadeus_token(session):
    try:
        credentials = base64.b64encode(
            f"{AMADEUS_CLIENT_ID}:{AMADEUS_CLIENT_SECRET}".encode("utf-8")
        ).decode("utf-8")
        async with session.post(
            "https://test.api.amadeus.com/v1/security/oauth2/token",
            headers={
                "Content-Type": "application/x-www-form-urlencoded",
                "Authorization": f"Basic {credentials}"
            },
            data={"grant_type": "client_credentials"}
        ) as response:
            if response.status == 200:
                token_data = await response.json()
                return token_data.get("access_token")
            else:
                error_response = await response.text()
                logger.error(f"Failed to fetch access token: {response.status}, Response: {error_response}")
                return None
    except Exception as e:
        logger.error(f"Error fetching access token: {e}")
        return None

async def fetch_hotels_and_flights(origem_iata, cidades, data_partida):
    print("origem IATA:", origem_iata)
    session = await get_session()
    access_token = await get_amadeus_token(session)
    results = []

    # Calculate check-in and check-out dates (e.g., 3-day stay)
    check_in_date = data_partida
    check_out_date = (datetime.strptime(data_partida, "%Y-%m-%d") + timedelta(days=3)).strftime("%Y-%m-%d")

    for cidade in cidades:
        cidade_iata = cidade["iata"]
        cidade_nome = cidade["nome"]

        # Step 1: Fetch hotel list by city
        logger.debug(f"Fetching hotels for city: {origem_iata} {cidade_iata}")
        hotel_list = await buscar_hoteis_por_cidade(session, cidade_iata, access_token)
        logger.debug(f"Hotel list for {origem_iata} {cidade_iata}: {hotel_list}")

        if not hotel_list:
            results.append({
                "city": cidade["nome"],
                "iata": cidade_iata,
                "hotels": "Nenhum hotel encontrado.",
                "flights": None
            })
            continue

        # Step 2: Fetch hotel offers for all hotels in the city (in batches)
        hotel_ids = [hotel["id"] for hotel in hotel_list]
        logger.debug(f"Fetching hotel offers for hotel IDs: {hotel_ids}")
        detailed_hotels = await buscar_ofertas_multiplos_hoteis(session, hotel_ids, access_token, check_in_date, check_out_date)
        logger.debug(f"Detailed hotels for {cidade_iata}: {detailed_hotels}")

        # Step 3: Filter the 5 hotels with the lowest fares
        if detailed_hotels:
            # Remove hotels with no price data
            detailed_hotels_with_price = [hotel for hotel in detailed_hotels if hotel["preco"] != "N/A"]
            # Sort by price (ascending)
            sorted_hotels = sorted(detailed_hotels_with_price, key=lambda x: float(x["preco"]))
            # Select the top 5
            top_5_hotels = sorted_hotels[:5]
        else:
            top_5_hotels = []

        # Step 4: Fetch flights
        logger.debug(f"Fetching flights for {origem_iata} -> {cidade_iata}")
        flights = await buscar_voos(session, origem_iata, cidade_iata, access_token, data_partida)
        logger.debug(f"Flights for {cidade_iata}: {flights}")

        results.append({
            "city": cidade_nome,
            "iata": cidade_iata,
            "hotels": top_5_hotels if top_5_hotels else "Nenhum hotel encontrado.",
            "flights": flights if flights else "Nenhum voo encontrado."
        })

    await session.close()
    logger.debug(f"Final results: {results}")
    return results

@app.route("/selecionar_cidade", methods=["POST"])
def selecionar_cidade():
    origem = request.form.get("origin")
    prompt_usuario = request.form.get("preferences")
    data_partida = request.form.get("date")
    selected_city_iata = request.form.get("selected_city")

    logger.info(f"Selected IATA code: {selected_city_iata}")
    logger.info(f"Origem: {origem}")
    logger.info(f"Prompt do usuário: {prompt_usuario}")
    logger.info(f"Data de partida: {data_partida}")

    if selected_city_iata:
        origem_iata = selected_city_iata
    else:
        resultados = buscar_iata_por_cidade(origem)
        if not resultados:
            return render_template("index.html", error=f"Não foi possível encontrar o código IATA para: {origem}", preferences=prompt_usuario, origin=origem, date=data_partida)
        if len(resultados) > 1:
            return render_template("selecionar_cidade.html", cidades=resultados, origem=origem, prompt_usuario=prompt_usuario, date=data_partida)
        origem_iata = resultados[0]["iata"]
        print("origem IATA: em selecionar_cidade", origem_iata)
        print("lista cidades em selecionar_cidades", cidades)

    if not data_partida:
        data_partida = get_default_departure_date()
        logger.info(f"Nenhuma data de partida fornecida. Usando data padrão: {data_partida}")
    elif not validate_date(data_partida):
        return render_template("index.html", error="Formato de data inválido. Use AAAA-MM-DD.", preferences=prompt_usuario, origin=origem, date=data_partida)

    sugestoes, cidades = gerar_sugestoes_viagem(prompt_usuario)
    logger.info(f"Sugestões geradas: {sugestoes}")
    logger.info(f"Cidades encontradas: {cidades}")

    results = asyncio.run(fetch_hotels_and_flights(origem_iata, cidades, data_partida))
    logger.info(f"Resultados da busca: {results}")

    # Buscar imagens para cada cidade
    imagens = {}
    for cidade in cidades:
        query = cidade["nome"]
        imagens[query] = buscar_imagens_pixabay(query)
#    print("Imagens encontradas:", imagens)  # Depuração
    return render_template("results.html", suggestions=sugestoes, results=results, imagens=imagens)


# Lista de lugares famosos para turismo
LUGARES_FAMOSOS = [
    "Paris", "New York", "Tokyo", "Rome", "London",
    "Barcelona", "Dubai", "Sydney", "Rio de Janeiro", "Machu Picchu", "Madrid", "Viena", "Africa", "Lisboa"
]

def buscar_imagem_aleatoria():
    lugar = random.choice(LUGARES_FAMOSOS)
    imagens = buscar_imagens_pixabay(lugar)
    return imagens[0] if imagens else None

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        origem = request.form.get("origin")
        prompt_usuario = request.form.get("preferences")
        data_partida = request.form.get("date")

        if not origem:
            return render_template("index.html", error="Por favor, forneça a cidade de origem.")

        resultados = buscar_iata_por_cidade(origem)
        if not resultados:
            return render_template("index.html", error=f"Nenhuma cidade encontrada para: {origem}. Procure não usar acentuação ou ~ !", preferences=prompt_usuario, origin=origem, date=data_partida)

        if len(resultados) > 1:
            return render_template("selecionar_cidade.html", cidades=resultados, origem=origem, prompt_usuario=prompt_usuario, data_partida=data_partida)

        origem_iata = resultados[0]["iata"]

        if not data_partida:
            data_partida = get_default_departure_date()
        elif not validate_date(data_partida):
            return render_template("index.html", error="Formato de data inválido. Use AAAA-MM-DD.", preferences=prompt_usuario, origin=origem, date=data_partida)

        sugestoes, cidades = gerar_sugestoes_viagem(prompt_usuario)
        results = asyncio.run(fetch_hotels_and_flights(origem_iata, cidades, data_partida))

        # Buscar imagens para cada cidade
        imagens = {}
        for cidade in cidades:
            query = cidade["nome"]
            imagens[query] = buscar_imagens_pixabay(query)

        return render_template("results.html", suggestions=sugestoes, results=results, imagens=imagens)

    # Para requisições GET, buscar uma imagem aleatória
    imagem_fundo = buscar_imagem_aleatoria()
    return render_template("index.html", imagem_fundo=imagem_fundo)
if __name__ == "__main__":
    print(f" * Running on {public_url}")
    app.run()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

 * Running on https://842c-34-87-58-213.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 18:46:57] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 18:46:58] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 18:48:06] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 18:48:06] "GET /static/HI9M.gif HTTP/1.1" 200 -


busco aventuras em locais com salto de asa delta ou parapaint  de montanhas e cachoeiras Liste 3 de destinos turísticos, incluindo: - Informe cidades com serviço regular de voos que sejam proximas às atracoes destacadas: Nome da Cidade e seu codigo IATA em parentesis: (exemplo: 'San Juan (SJU)'). - Principais atrações e atividades. use no maximo 1000 palavras de forma que resuma bem o pedido do usuario:

1. Interlaken, Suíça (ZRH)
   - Principais atrações: Lago Thun, Jungfrau, Schynige Platte, Harder Kulm, e Aventura de Canyon-Swing.
   - Atividades: Salto de asa delta, parapente, trekking, e visitas a cachoeiras.

2. Queenstown, Nova Zelândia (ZQN)
   - Principais atrações: Fiordes de Milford e Doubtful, Cascata de Kawarau, e Remarkables Mountain Range.
   - Atividades: Salto de asa delta, parapente, trekking, e visitas a cachoeiras.

3. Cusco, Peru (CUZ)
   - Principais atrações: Machu Picchu, Cidade de Cusco, Sacred Valley, e Ollantaytambo.
   - Atividades: Trekking, visitas a cacho

ERROR:__main__:Failed to fetch hotel offers: 400, Response: {"errors":[{"code":3664,"title":"NO ROOMS AVAILABLE AT REQUESTED PROPERTY","status":400,"detail":"Provider Error - NO ROOMS AVAILABLE AT REQUESTED PROPERTY","source":{"parameter":"hotelIds=YXZRH313,YXZRHALE,YXZRHAST,YXZRHCHR,YXZRHEMA,YXZRHH07,YXZRHHSC,YXZRHILL,YXZRHKIN,YXZRHLIH,YXZRHLIM,YXZRHORH,YXZRHRHR,YXZRHSBI,YXZRHSPI,YXZRHSTG,YXZRHZGK"}},{"code":3289,"title":"RATE NOT AVAILABLE FOR REQUESTED DATES","status":400,"detail":"Provider Error - RATE NOT AVAILABLE FOR REQUESTED DATES","source":{"parameter":"hotelIds=YXZRHH06"}},{"code":1257,"title":"INVALID PROPERTY CODE","status":400,"detail":"Provider Error - INVALID PROPERTY CODE","source":{"parameter":"hotelIds=YXZRH435,YXZRH577,YXZRHCGS,YXZRHHOT,YXZRHHPK,YXZRHHPL,YXZRHHVT,YXZRHMAR,YXZRHRVA,YXZRHSAZ,YXZRHSEN,YXZRHSTH"}}]}


...

ERROR:__main__:Failed to fetch hotel offers: 400, Response: {"errors":[{"code":4070,"title":"UNABLE TO PROCESS - CONTACT HELP DESK","status":400,"detail":"Provider Error - UNABLE TO PROCESS - CONTACT HELP DESK","source":{"parameter":"hotelIds=HIVCE9F8"}},{"code":1257,"title":"INVALID PROPERTY CODE","status":400,"detail":"Provider Error - INVALID PROPERTY CODE","source":{"parameter":"hotelIds=HOVCECEF,HSVCEABB,HSVCEABW,HSVCEABX,HSVCEACB,HSVCEACD,HSVCEACG,HSVCEACR,HSVCEADI,HSVCEADY,HSVCEAEI,HSVCEAFB,HSVCEAFR,HSVCEAGT,HSVCEAGW,HSVCEAHL,HSVCEAHZ,HSVCEAIL,HSVCEAIP,HSVCEAJK,HSVCEAJM,HSVCEAKB,HSVCEALC,HSVCEALF,HSVCEANG,HSVCEANK,HSVCEAPK,HSVCEAPT,HSVCEAQB,HSVCEARF,HSVCEARI,HSVCEARJ,HSVCEARO,HSVCEASP,HSVCEATG,HSVCEATT,HSVCEATX,HSVCEAWI,HSVCEAWJ,HSVCEAWL"}},{"code":3664,"title":"NO ROOMS AVAILABLE AT REQUESTED PROPERTY","status":400,"detail":"Provider Error - NO ROOMS AVAILABLE AT REQUESTED PROPERTY","source":{"parameter":"hotelIds=HAVCEALV,HAVCECAD,HAVCEH13,HAVCENID,HSVCEAEF,HSVCEAGB,HSVCEALO,H

[{'id': 'ARFLRFIR', 'name': 'AC BY MARRIOTT HOTEL FIRENZE'}, {'id': 'BWFLR309', 'name': 'BEST WESTERN HOTEL RIVER'}, {'id': 'BWFLR267', 'name': 'BW HOTEL PALAZZO OGNISSANTI'}, {'id': 'BWFLR158', 'name': 'BEST WESTERN GRAND HOTEL ADRIATICO'}, {'id': 'CHFLRHMT', 'name': 'HOTEL MARTELLI'}, {'id': 'CHFLRHRH', 'name': 'RESIDENCE HILDA'}, {'id': 'CHFLRH38', 'name': 'HOTEL PRIVILEGE'}, {'id': 'CHFLRH49', 'name': 'HOTEL ANNABELLA'}, {'id': 'CHFLRHCE', 'name': 'HOTEL I CEDRI'}, {'id': 'DHFLRANY', 'name': 'C-HOTELS THE STYLE FLORENCE'}, {'id': 'DSFLRCON', 'name': 'CONTINENTALE'}, {'id': 'EPFLR625', 'name': 'RELAIS VILLA BELPOGGIO'}, {'id': 'EPFLRMON', 'name': 'UNA PALAZZO MANNAIONI'}, {'id': 'EUFLRVTO', 'name': 'VILLA TOLOMEI HOTEL & RESORT'}, {'id': 'FGFLRH01', 'name': 'HOTEL BALESTRI'}, {'id': 'FGFLRBIL', 'name': 'AFFITTACAMERE CASA BILLI'}, {'id': 'GAFLRAUG', 'name': 'GALLERY HOTEL ART'}, {'id': 'GIFLR424', 'name': 'HILTON GRDN INN FLORENCE NOVOLI'}, {'id': 'GZFLROLI', 'name': 'HOTEL OLIMPIA'

ERROR:__main__:Failed to fetch hotel offers: 400, Response: {"errors":[{"code":3289,"title":"RATE NOT AVAILABLE FOR REQUESTED DATES","status":400,"detail":"Provider Error - RATE NOT AVAILABLE FOR REQUESTED DATES","source":{"parameter":"hotelIds=YXFLR444,YXFLRBAH,YXFLRGIG,YXFLRH06,YXFLRTDB"}},{"code":1257,"title":"INVALID PROPERTY CODE","status":400,"detail":"Provider Error - INVALID PROPERTY CODE","source":{"parameter":"hotelIds=YXFLR154,YXFLR424,YXFLR482,YXFLR834,YXFLR842,YXFLRBN5,YXFLRBRU,YXFLRH03,YXFLRH14,YXFLRH25,YXFLRH27,YXFLRH29,YXFLRH44,YXFLRH56,YXFLRHBE,YXFLRHDC,YXFLRHDR,YXFLRHMG,YXFLRKUR,YXFLRROV,YXFLRSIH,YXFLRSTA,YXFLRTOR"}},{"code":3664,"title":"NO ROOMS AVAILABLE AT REQUESTED PROPERTY","status":400,"detail":"Provider Error - NO ROOMS AVAILABLE AT REQUESTED PROPERTY","source":{"parameter":"hotelIds=YXFLRC20,YXFLRHDF,YXFLRNAZ,YXFLRPOG,YXFLRVDP"}}]}
INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 19:38:56] "POST /selecionar_cidade HTTP/1.1" 200 -


Imagens encontradas: {'Roma': ['https://pixabay.com/get/g7eaa4e36d7999e294cc06e4b60e819fd1f34c84729d04fdffba54363f8b2ac5d8fa0279c2885eee1cca53e6596ec6ddc79fc45a26c3285567d5c291cfc477273_1280.jpg', 'https://pixabay.com/get/g2a29fe7b2c0ff10d6fc2593094b5b0674bc30a638e6587134c99a9268fa4edf639de6e2126bbe246d7a63313730f87f0182233fd744f943f82a1ae466fd1ff95_1280.jpg', 'https://pixabay.com/get/g677ab634595962ef0634b25beda8d8f05ffc2901374f5cc1af0df5e9ae2ea902b84f4ed8b37f3dd2ecd1118bc9fefeb96684744dcf9af40931b860baa6216fb7_1280.jpg', 'https://pixabay.com/get/g5ac7ec56873386f7405743ad4919fd4c148442bce0d27ad5df0684269b5f92d3e0d98c48949817c9cf3eddefb4d31fb64234b28c7dc95b89f1321c9a39780694_1280.jpg', 'https://pixabay.com/get/g2bccced962dfa7e0e30112b5ab07b4f4aa07b715364eaa1d24cfaaffc01d5ad161b5ebd22114f2d661d613170b37c643e70889ce9a0ff37d9d0b466201592769_1280.jpg'], 'Veneza': ['https://pixabay.com/get/g95d873337c38951c09d56a88fe8d76fec4c412e52cfc01a8041b8d87533ccdd08ff7ad366b066312327847d7c2a87e63b22ab

INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 19:40:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 19:43:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/Mar/2025 19:43:10] "GET /favicon.ico HTTP/1.1" 404 -
